In [45]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [2]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [5]:
def apply_exposure(image, factor=1.5):
    exposure_image = ImageEnhance.Brightness(image).enhance(factor)
    return exposure_image

# Ensure the target directory exists
exposed_dir = 'dataset/exposed'
if not os.path.exists(exposed_dir):
    os.makedirs(exposed_dir)

# Specify the path to the original images
original_dir = 'dataset/original'
total_images = 20

# Create exposed images and save
for i in range(1, total_images + 1):
    img_path = f'{original_dir}/{i:02d}.jpg'
    img = Image.open(img_path).convert('RGB')
    for j in range(20):  # Generate 20 exposed images for each original image
        exposed_img = apply_exposure(img)
        exposed_img.save(f'{exposed_dir}/exposed_{i:02d}_{j}.jpg')

In [22]:
# Define data paths
original_dir = 'dataset/original'
exposed_dir = 'dataset/exposed'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'exposed']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(exposed_dir, os.path.join(train_dir, 'exposed'), os.path.join(validation_dir, 'exposed'))

In [31]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 340 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [30]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['original', 'exposed']
Updated validation directories: ['original', 'exposed']


In [39]:
print("Directories in dataset:", os.listdir(dataset_dir))

for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):
        print(f"Contents of {category}:", os.listdir(category_path))

Directories in dataset: ['validation', 'train']
Contents of validation: ['original', 'exposed']
Contents of train: ['original', 'exposed']


In [33]:
import shutil
import os

directories_to_remove = ['original', 'exposed']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Removed directory: dataset/original
Removed directory: dataset/exposed
Updated contents of the dataset directory: ['validation', 'train']


In [46]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

Found 344 images belonging to 2 classes.
Found 86 images belonging to 2 classes.


In [47]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 193s 5s/step - loss: 9.9559 - accuracy: 0.5733 - val_loss: 13.0022 - val_accuracy: 0.7200
Epoch 2/10
15/15 [==============================] - 74s 5s/step - loss: 21.1139 - accuracy: 0.6133 - val_loss: 5.4439 - val_accuracy: 0.8000
Epoch 3/10
15/15 [==============================] - 67s 5s/step - loss: 9.2858 - accuracy: 0.6533 - val_loss: 0.3035 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 67s 4s/step - loss: 11.2006 - accuracy: 0.6800 - val_loss: 7.4084 - val_accuracy: 0.7200
Epoch 5/10
15/15 [==============================] - 74s 5s/step - loss: 10.7514 - accuracy: 0.7200 - val_loss: 10.1580 - val_accuracy: 0.6800
Epoch 6/10
15/15 [==============================] - 79s 5s/step - loss: 8.2324 - accuracy: 0.5946 - val_loss: 7.2900 - val_accuracy: 0.6400
Epoch 7/10
15/15 [==============================] - 67s 4s/step - loss: 7.0706 - accuracy: 0.7200 - val_loss: 10.1505 - val_accuracy: 0.6000
Epoch 8/10
15